In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from google.cloud.firestore_v1.document import DocumentReference
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)


db = firestore.client()


def load_json_file(filename):
    with open(filename) as fp:
        content = json.load(fp)
    return content


In [2]:
def process_reference(reference: DocumentReference):
    return reference.path

def process_array(array: list):
    return [process_reference(item) if type(item) == DocumentReference else item for item in array]

def process_dict(input_dict):
    output_dict = {}
    for key, value in input_dict.items():
        if type(value) == DocumentReference:
            output_dict[key] = process_reference(value)
        elif isinstance(value, dict):
            output_dict[key] = process_dict(value)
        elif isinstance(value, list):
            output_dict[key] = process_array(value)
        elif isinstance(value, datetime):
            output_dict[key] = value.isoformat()
        else:
            output_dict[key] = value
    return output_dict

In [3]:
def split_string_ref(str_fullref: str):
    collection, key = str_fullref.split("/")
    return collection, key

# Carrega os dados que já estão salvos

In [4]:
items = load_json_file("json1123/itens.json")
volumes = load_json_file("json1123/volumes.json")

In [5]:
def load_data_from_fullref(fullref: str):
    collection, key = split_string_ref(fullref)
    if collection == "volumes":  
        return volumes[key]
    elif collection == "items":
        return items[key]
    

# Nossos itens que já foram mensurados

In [6]:
itens_mensurados = list(filter(lambda x: re.match(r".*\(AxCxL\)", x["detalhes"]["medidas"]) != None, items.values()))
itens_mensurados_keys = [x["key"] for x in itens_mensurados]

In [9]:
# filtra os itens que estão volumados em algum volume que está na lista de mobilia errada
itens_mensurados_volumados_errados = list(filter(lambda x: x["meta"]["volume"] != None and x["meta"]["volume"].split("/")[1] in mobilia_errada_codigo, itens_mensurados))

In [10]:


def clone_original_volume(item: dict) -> dict:
    """Recebe um item e retorna um volume com os dados do item e o item como único item do volume"""
    volume_fullref = item["meta"]["volume"]
    volume_copy = load_data_from_fullref(volume_fullref).copy()
    volume_copy["items"] = [ db.document(f"items/{item['key']}") ]
    for prop in ["responsavel", "origem", "destino", "localizacao_atual"]:
        volume_copy[prop] = db.document(volume_copy[prop])
    volume_copy["data_criacao"] = datetime.now()
    volume_copy["codigo"] = f"V{item['key']}"
    volume_copy["medidas"] = extract_medidas(item)
    volume_copy["observacao"] = "Criado pelo sistema. Volume original: " + volume_fullref
    return volume_copy

def clone_original_volume_error(item: dict) -> dict:
    """Recebe um item e retorna um volume com os dados do item e o item como único item do volume"""
    volume_fullref = item["meta"]["volume"]
    volume_copy = load_data_from_fullref(volume_fullref).copy()
    volume_copy["items"] = [ db.document(f"items/{item['key']}") ]
    for prop in ["responsavel", "origem", "destino", "localizacao_atual"]:
        volume_copy[prop] = db.document(volume_copy[prop])
    volume_copy["data_criacao"] = datetime.now()
    volume_copy["codigo"] = f"V{item['key']}"
    volume_copy["medidas"] =  { "a": 1, "c": 2, "l": 3}
    volume_copy["observacao"] = "Criado pelo sistema. PRECISA AJUSTAR AS MEDIDAS. Volume original: " + volume_fullref 
    return volume_copy

def soft_delete_volume(volume_fullref: str):
    volume_db = db.document(volume_fullref)
    volume_db.update({"deleted": True})
    

In [11]:
def extract_medidas_loose(descricao: str):
    """
    Extracts the measures from a string
    :param descricao: string to extract the measures from
    :return: a tuple of integers with the extracted measures
    """ 
    return tuple(map(int, re.findall(r'[xX]*(\d+).*?[Xx]', descricao)))

def extract_medidas(item: dict) -> dict:
    medidas = item["detalhes"]["medidas"]
    match = re.search(r"(\d+) x (\d+) x (\d+)", medidas)
    if match == None: 
        raise Exception(f"Não foi possível extrair as medidas do item {item['key']}")
    else:
        medidas_groups = match.groups()
        medidas_int = list(map(lambda x: int(x), medidas_groups))
        return dict(zip(["a", "c", "l"], medidas_int)) 

In [12]:

def main_restructure_volumes(lista_volumes: list):
    NEW_VOLUMES = []
    NEW_VOLUMES_REF = []

    output = {'certos': 0, 'errados': 0}
    for mobilia in lista_volumes:
        errors = 0
        volume_fullref = f"volumes/{mobilia['codigo']}"
        print(f"Processando o volume {volume_fullref}")
        for item in mobilia["items"]:
            NEW_VOLUMES_REF.append(volume_fullref)
            item_dict = load_data_from_fullref(item)
            new_volume = None
            if isinstance(item_dict["key"],int):
                print(f"\t{bcolors.OKBLUE}O item {item_dict['key']} é patrimoniado.{bcolors.ENDC}")
            else:
                print(f"\t{bcolors.WARNING}O item {item_dict['key']} não é patrimoniado.{bcolors.ENDC}")
            try:
                new_volume = clone_original_volume(item_dict)
                print(f"\t\t{bcolors.OKGREEN}O item {item_dict['key']} foi volumado com sucesso.{bcolors.ENDC}")
                output['certos'] += 1
            except Exception as e:
                output['errados'] += 1
                errors += 1
                new_volume = clone_original_volume_error(item_dict)
                print(f"\t\t{bcolors.FAIL}O item {item_dict['key']} foi volumado com erros: não foi possível determinar suas medidas.{bcolors.ENDC}")
            NEW_VOLUMES.append(new_volume)
        if errors == 0:
            print(f"{bcolors.OKGREEN}Tudo certo com o volume {volume_fullref}{bcolors.ENDC}")
        else:
            print(f"{bcolors.FAIL}O volume {volume_fullref} tem {errors} erros{bcolors.ENDC}")
    return NEW_VOLUMES,NEW_VOLUMES_REF, output

In [13]:
def batch_write(collection: str, dictarray: list[dict], key_id: str | None = None):
    batch = db.batch()
    collection_ref = db.collection(collection)
    k = 1
    for i in range(0, len(dictarray), 500):
        chunk = dictarray[i:i+500]
        for document in chunk:
            if key_id:
                doc_ref = collection_ref.document(str(document[key_id]))
                batch.set(doc_ref, document)
        print(f"{collection}: chunk {k} ({len(chunk)} documents)")
        k += 1
        batch.commit()

In [14]:
def batch_soft_delete(refs_list: list): 
    batch = db.batch() 
    k = 1
    for i in range(0, len(refs_list), 500):
        chunk = refs_list[i:i+500]
        for full_ref in chunk: 
            doc_ref = db.document(full_ref)
            batch.update(doc_ref, {'deleted': True})
        print(f"chunk {k} ({len(chunk)} documents)")
        k += 1
        batch.commit()

In [15]:
1=2

SyntaxError: cannot assign to literal here. Maybe you meant '==' instead of '='? (3293032332.py, line 1)

# Regra 1

Vamos procurar por mobília que tem mais de 1 item

In [ ]:
mobilia_errada = list(filter(lambda x: x["categoria"] == "Mobília" and len(x["items"]) > 1 and x["origem"] !=  "ambientes/A001", volumes.values())) 

In [ ]:
mobilia_errada_codigo = [x["codigo"] for x in mobilia_errada]

## Teste 1

In [ ]:
volume_teste = volumes['cute3qhONPhHp3D0rWlh']

In [ ]:
batch_write("volumes", NEW_VOLUMES, "codigo")

volumes: chunk 1 (500 documents)
volumes: chunk 2 (496 documents)


# Regra 2

In [ ]:
escritorio_preerrado = list(filter(lambda x: x["categoria"] == "Material de Escritório" and len(x["items"]) > 1, volumes.values())) 

In [ ]:
items_escritorio_preerrado = [ [ load_data_from_fullref(x) for x in volume["items"] ] for volume in escritorio_preerrado ]

In [ ]:
from functools import reduce

In [ ]:
filtro_items_escrits = [ reduce(lambda x,y: x or y, map(lambda x: x['key'] in itens_mensurados_keys, x))  for x in items_escritorio_preerrado ] 

In [ ]:
for mask, volume in zip(filtro_items_escrits, escritorio_preerrado):
    if mask:
        print(f"{bcolors.WARNING}O volume {volume['codigo']} tem itens mensurados{bcolors.ENDC}")
    else:
        print(f"{bcolors.OKGREEN}O volume {volume['codigo']} não tem itens mensurados{bcolors.ENDC}")

O volume 0X5yR4FsvkAy5QB7ELKB não tem itens mensurados
O volume 1KNViKcI6mOfQ6bpi5Jv tem itens mensurados
O volume 2QDkly2BfP0VmNoaFBIa tem itens mensurados
O volume 31QI0ZC2Kg7EvV1V7z1a tem itens mensurados
O volume 48yWUZsJlCjpB9vP5J3Y tem itens mensurados
O volume 4dwc1D19ramci586997J tem itens mensurados
O volume 5Z4tHJnXIheImmU5JZ93 tem itens mensurados
O volume 7FEZixWuEaGBh7zrUoN5 tem itens mensurados
O volume 7TsTXQB1LDcO7X1EPUbL não tem itens mensurados
O volume 7yFGEiEbzBYxFh98Uwbp tem itens mensurados
O volume 8Z4JuNeQin5pTSQR9Gtg tem itens mensurados
O volume 8zzJh9gjzLuHy2OBCp3V tem itens mensurados
O volume ANH5B7eQ9WFeOo5gwbSx tem itens mensurados
O volume AzXPbdoMjjFwoNNvTTDC tem itens mensurados
O volume CZPmFBtmL0VgSuqlaUyh tem itens mensurados
O volume CoC8ON5taCdB3YNvcmXF tem itens mensurados
O volume E0C8cijlBJi20oY9rDh2 tem itens mensurados
O volume E4UggbXSg3MkqXm9BNd2 tem itens mensurados
O volume EjvaPSBpzc7mPe6pwdnE não tem itens mensurados
O volume FkWuYp338V

In [ ]:
escritorio_errado = [ volume for (mask, volume) in zip(filtro_items_escrits, escritorio_preerrado) if mask ] 

In [ ]:
NEW_ESCRITORIOS_VOLUMES, NEW_ESCRITORIOS_VOLUMES_REFS, OUTPUT = main_restructure_volumes(escritorio_errado)

Processando o volume volumes/1KNViKcI6mOfQ6bpi5Jv
	O item 338138 é patrimoniado.
		O item 338138 foi volumado com erros: não foi possível determinar suas medidas.
	O item 338125 é patrimoniado.
		O item 338125 foi volumado com sucesso.
	O item 360020 é patrimoniado.
		O item 360020 foi volumado com sucesso.
	O item 359963 é patrimoniado.
		O item 359963 foi volumado com sucesso.
O volume volumes/1KNViKcI6mOfQ6bpi5Jv tem 1 erros
Processando o volume volumes/2QDkly2BfP0VmNoaFBIa
	O item 317075 é patrimoniado.
		O item 317075 foi volumado com erros: não foi possível determinar suas medidas.
	O item 310307 é patrimoniado.
		O item 310307 foi volumado com sucesso.
	O item 364260 é patrimoniado.
		O item 364260 foi volumado com erros: não foi possível determinar suas medidas.
	O item 359965 é patrimoniado.
		O item 359965 foi volumado com sucesso.
	O item 314934 é patrimoniado.
		O item 314934 foi volumado com erros: não foi possível determinar suas medidas.
	O item 370806 é patrimoniado.
		

In [ ]:
OUTPUT

{'certos': 162, 'errados': 137}

In [ ]:
NEW_ESCRITORIOS_VOLUMES_TO_ARRAY = [ process_dict(value) for value in NEW_ESCRITORIOS_VOLUMES ]



with open("json1123/mobilia_save.json", "w") as fp:
    json.dump({'new_volumes': NEW_ESCRITORIOS_VOLUMES_TO_ARRAY, 'old_keys': NEW_ESCRITORIOS_VOLUMES_REFS},fp, ensure_ascii=False, indent=2)

In [ ]:
batch_soft_delete(list(set(NEW_ESCRITORIOS_VOLUMES_REFS)))

chunk 1 (50 documents)


In [ ]:
batch_write("volumes", NEW_ESCRITORIOS_VOLUMES, "codigo")

volumes: chunk 1 (299 documents)


In [ ]:
500 + 496 + 299

1295